In [1]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import numpy as np
from datetime import timedelta, datetime
import folium
import warnings
import pickle
import sys

warnings.filterwarnings('ignore')

print("Geopandas has version {}".format(gpd.__version__))
print("Movingpandas has version {}".format(mpd.__version__))

Geopandas has version 0.13.2
Movingpandas has version 0.17.1


In [2]:
# add paths for modules
sys.path.append('../models')
# import modules
import visualize
from maritime_traffic_network import MaritimeTrafficNetwork

In [3]:
# load network from pickle
model = '202204_waypoints_DP10_HDBSCAN12_tromso_full_UTM'
model_path = '../../models/networks/best_networks/' + model + '.obj'
fileObj = open(model_path, 'rb')
network = pickle.load(fileObj)
fileObj.close()
network.hyperparameters

{'Data': '../../data/processed/202204_points_tromso_cleaned_meta_full_dualSplit_2.parquet',
 'DP_tolerance': 10,
 'clustering_method': 'HDBSCAN',
 'clustering_metric': 'mahalanobis',
 'clustering_min_samples': 12,
 'clustering_min_cluster_size': 12,
 'clustering_eps': 0,
 'clustering_metric_V': array([[1.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00],
        [0.e+00, 1.e+00, 0.e+00, 0.e+00, 0.e+00],
        [0.e+00, 0.e+00, 1.e-02, 0.e+00, 0.e+00],
        [0.e+00, 0.e+00, 0.e+00, 1.e-02, 0.e+00],
        [0.e+00, 0.e+00, 0.e+00, 0.e+00, 5.e-04]]),
 'graph_generation_max_distance': 10,
 'graph_generation_max_angle': 45}

In [4]:
# merge stop points
network.merge_stop_points(max_speed=2)

In [5]:
# prune
network.prune_graph(1)

Pruning...
------------------------
Pruned Graph:
Number of nodes: 565 (1 isolated)
Number of edges: 2181
------------------------


In [6]:
# plot network and basemap
network.prune_graph(100)
map = network.map_graph(pruned=True)
network.prune_graph(1)
# add some trajectories
all_trajectories = network.trajectories
selection = [1]
mmsis = network.gdf.mmsi.unique()[selection]
trajectories = all_trajectories.filter('mmsi', mmsis.tolist())
for mmsi in mmsis:
    trajectory = all_trajectories.get_trajectory(mmsi)
    path_df, evaluation_results = network.trajectory_to_path(trajectory)
    try:
        map = trajectory.to_line_gdf()[['geometry', 'mmsi', 'skipsgruppe']].explore(m=map, style_kwds={'weight':3, 'color':'black', 'opacity':1}, 
                                                                      name=mmsi+' trajectory')
        map = path_df.explore(m=map, name=mmsi+' edge_sequence ', style_kwds={'weight':3, 'color':'yellow', 'opacity':1})
    except:
        print(mmsi+': no path found')

folium.LayerControl().add_to(map)
#map

Pruning...
------------------------
Pruned Graph:
Number of nodes: 565 (1 isolated)
Number of edges: 1453
------------------------
Pruning...
------------------------
Pruned Graph:
Number of nodes: 565 (1 isolated)
Number of edges: 2181
------------------------
Intersections found: [28, 470, 465, 31, 472, 32, 4, 467, 187, 343, 188, 302, 308, 309, 312, 313, 310, 311, 316, 317, 318, 319, 300, 298, 292, 532, 541, 548, 560, 563, 564, 565, 562, 561, 556, 558, 559, 557, 551, 552, 544, 535, 536, 525, 526, 524, 504, 464, 462, 450, 365, 429, 433, 434, 435, 427, 415]
Iterating through edge sequences
Edge sequence: [28, 470]
Sequence: [28, 470]
Mean distance: 27.442932778199452
Edge sequence: [28, 475, 470]
Sequence: [28, 475, 470]
Mean distance: 65.21812528230342
Edge sequence: [28, 451, 475, 470]
Sequence: [28, 451, 475, 470]
Mean distance: 93.16660323068535
Edge sequence: [28, 451, 470]
Sequence: [28, 451, 470]
Mean distance: 102.29761397911145
Edge sequence: [28, 31, 4, 496, 493, 470]
Sequenc

In [7]:
map.save('../../reports/maps/'+model+'.html')

In [8]:
evaluation_results

,mmsi,mean_dist,median_dist,max_dist,distances,fraction_covered,length ratio,message
0,210055000_4_2022-04-01 04:09:32,96.271792,77.804769,314.146688,"[16.91894639703674, 15.75231666641875, 14.5854...",1.0,0.999532,success
